In [ ]:
connections = {
    "Home": {"Yellow": 270, "Green": 90},
    "Yellow": {"Red": 270, "Home": 90},
    "Green": {"Home": 270, "Blue": 90},
    "Red": {"L_purple": 180, "Yellow": 90},
    "Blue": {"L_orange": 180, "Green": 270},
    "L_orange": {"Blue": 0, "J_L_orange": 180},
    "L_purple": {"Red": 0, "J_L_purple": 180},
    "J_L_orange": {"L_orange": 0, "Ramp": 270},
    "J_L_purple": {"L_purple": 0, "Ramp": 90},
    "Ramp": {"U_T": 0, "J_L_orange": 90, "J_L_purple": 270},
    "U_T": {"J_U_orange": 90, "Ramp": 180, "J_U_purple": 270},
    "J_U_orange": {"U_orange": 180, "U_T": 270},
    "J_U_purple": {"U_purple": 180, "U_T": 90},
    "U_orange": {"J_U_orange": 0},
    "U_purple": {"J_U_purple": 0}
    }

path_det = {
    "Home": 5,
    "Yellow": 6,
    "Green": 4,
    "Red": 7,
    "Blue": 3,
    "L_purple": 8,
    "L_orange": 2,
    "Ramp": None
}

class State:
    def __init__(self, loc, orien):
        self.loc = loc
        self.orien = orien
        self.destination = None
        self.bay = None
        self.remaining_boxes = ["Red", "Blue", "Green", "Yellow"]

global state
state = State("Home", 180)

def mini_path_find(start, dest, min_max):
    queue = [start]
    while start != dest:
        branches = list(connections[start].keys())
        if min_max == "min":
            start = min(branches, key = lambda branches: path_det[branches])
        elif min_max == "max":
            start = max(branches, key = lambda branches: path_det[branches])
            
        queue.append(start)
    return queue

def path_find(start, dest):
    if dest == "L_orange":
        return mini_path_find(start, dest, "min")
    elif dest == "L_purple":
        return mini_path_find(start, dest, "max")
    elif dest == "U_purple" or dest == "U_orange":
        if start == "Red" or start == "Yellow":
            queue = mini_path_find(start, "L_purple", "max")
            queue.append("J_L_purple")
        elif start == "Green" or start == "Blue":
            queue = mini_path_find(start, "L_orange", "min")
            queue.append("J_L_orange")
        else:
            raise Exception("error 2, elifs ran out")
        queue.append("Ramp")
        queue.append("U_T")
        queue.append("J_" + dest)
        queue.append(dest)
        return queue
        
    else:
        raise Exception("error 1, elifs ran out")
    
    

In [6]:
print(path_find("Red", "L_orange"))

['Red', 'Yellow', 'Home', 'Green', 'Blue', 'L_orange']


In [ ]:
SMALL = 10
MEDIUM = 40
STEP_FWD = 40

def fwd_until_junc():
    pass

def clockwise():
    pass

def anticlockwise():
    pass

def load_fork():
    pass

def unload_fork():
    pass

def light_readings():
    pass

def fwd_until_black():
    pass

def fwd_until_box():
    pass

def fwd(distance):
    pass

def is_RHS_lidar_pos():
    pass

def is_LHS_lidar_pos():
    pass

def rvs(distance):
    pass

def read_qr():
    pass

def count_unload_return():
    counter = 0
    if state.destination == "U_purple" or state.destination == "L_orange":
        dest_general = 7 - state.bay
    else:
        dest_general = state.bay
    while counter < dest_general:
        fwd_until_junc()
        counter += 1
    turn_unload_return(dest_general)

def execute_travel(route):
    destination = route[-1]
    output = route
    next_node = route.pop(0)
    while route:
        current_node = next_node
        next_node = route.pop(0)
        print(current_node, next_node)
        wanted_orien = connections[current_node][next_node]
        #clockwise is positive
        turn_angle = wanted_orien - state.orien
        if turn_angle == 90 or turn_angle == -270:
            clockwise()
            state.orien = (state.orien + 90) % 360
        elif turn_angle == -90 or turn_angle == 270:
            anticlockwise()
            state.orien = (state.orien - 90) % 360
        elif turn_angle == 180 or turn_angle == -180:
            clockwise()
            clockwise()
            state.orien = (state.orien + 180) % 360
        if {current_node, next_node} in [{"Red", "L_orange"}, {"Blue", "L_purple"}] and not destination in ["L_purple", "L_orange"]:
            for i in range(7):
                fwd_until_junc()
                fwd(SMALL)
        fwd_until_junc()
        state.loc = next_node
    
    
    print(turn_angle)

def return_to_color(path_fwd):
    path_rvs = path_fwd[::-1]
    while path_rvs[-2] in ["Red", "Yellow", "Green", "Blue"]:
        path_rvs.pop()
    execute_travel(path_rvs)


def phase_1_find_box():

    #starting at either Red or Blue pointing to 0 degrees, make the robot go to the nearest bay with a box and turn to face the box

    color_order = ["Red", "Yellow", "Green", "Blue"]
    if state.loc == "Red":
        i = 0
        while not state.loc in state.remaining_boxes:
            if i == 0:
                clockwise()
            fwd_until_junc()
            color_order.pop(0)
            state.loc = color_order(0)
            i += 1
        if i != 0:
            anticlockwise()

    elif state.loc == "Blue":
        color_order = color_order[::-1]
        i = 0
        while not state.loc in state.remaining_boxes:
            if i == 0:
                anticlockwise()
            fwd_until_junc()
            color_order.pop(0)
            state.loc = color_order(0)
            i += 1
        if i != 0:
            clockwise()
    else:
        raise Exception("error sth, elifs ran out")

def go_in_for_the_box_readqr():
    #with the bot facing the box, go stright in for the box
    fwd(SMALL)
    fwd_until_junc()
    fwd_until_box()
    destination, bay = read_qr()
    load_fork()
    if state.loc == "Red" or state.loc == "Yellow":
        clockwise()
        clockwise()
    elif state.loc == "Blue" or state.loc == "Green":
        anticlockwise()
        anticlockwise()

    fwd_until_junc()
    state.orien = 180
    state.remaining_boxes.remove(state.loc)
    return destination, bay

def phase_2_detect_boxes():

    color_order = ["Red", "Yellow", "Green", "Blue"]
    if state.loc == "Red":
        clockwise()
        while not is_LHS_lidar_pos():
            color_order.pop(0)
            if not color_order:
                print("NO BOXES DETECTED")
                anticlockwise()
                phase_2_detect_boxes()
            fwd_until_junc()
            state.loc = color_order(0)
        anticlockwise()

    elif state.loc == "Blue":
        color_order = color_order[::-1]
        anticlockwise()
        while not is_RHS_lidar_pos():
            color_order.pop(0)
            if not color_order:
                print("NO BOXES DETECTED")
                anticlockwise()
                phase_2_detect_boxes()
            fwd_until_junc()
            state.loc = color_order(0)
        clockwise()
    else:
        raise Exception("error sth, elifs ran out")


def turn_unload_return(bay):
    if state.destination == "U_purple" or state.destination == "L_orange":
        clockwise()
    elif state.destination == "L_purple" or state.destination == "U_orange":
        anticlockwise()
    else:
        raise Exception("error 3, elifs ran out")
    fwd_until_black()
    unload_fork()

    rvs(MEDIUM)
    clockwise()
    clockwise()
    fwd_until_junc()
    if state.destination == "U_purple" or state.destination == "L_orange":
        anticlockwise()
    elif state.destination == "L_purple" or state.destination == "U_orange":
        clockwise()
    else:
        raise Exception("error 3, elifs ran out")
    state.orien = 0
    fwd(bay*STEP_FWD)
    #this is a very fragile part of the code, the bot has to turn quite accurately

def start_at_yellow():
    fwd(MEDIUM)
    fwd_until_junc()
    clockwise()
    fwd_until_junc()
    clockwise()
    state.loc = "Yellow"
    state.orien = 0








In [ ]:

for q in range(4):
    if q == 0:
        start_at_yellow()
    else:    
        phase_1_find_box()
    state.destination, state.bay = go_in_for_the_box_readqr()
    route = path_find(state.loc, state.destination)
    execute_travel(route)
    count_unload_return()
    return_to_color(route)
    

for p in range(90):
    phase_2_detect_boxes()
    state.destination, state.bay = go_in_for_the_box_readqr()
    route = path_find(state.loc, state.destination)
    execute_travel(route)
    count_unload_return()
    return_to_color(route)
    
    

